In [59]:
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
from geopy.geocoders import Nominatim

## Geotagging the scraped Data

In [60]:
df = pd.read_csv('all_reviews.csv')
locations_df = pd.read_csv('locations_geotagged.csv')
beer_df = pd.read_csv('beer_deets.csv')

In [61]:
df

,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit
...,...,...,...,...,...,...
29073,Miller High Life,4.0,+2.3,Miller Brewing Company (Molson Coors),"People can trash this beer all they want, but ...",moomin
29074,New Glarus Spotted Cow,4.7,+1.6,New Glarus Brewing Company,"A classic delicious Wisconsin beer, maybe my f...",moomin
29075,Vibrant Forest Stone Sap - Juicy India Pale Ale,1.9,-1.6,Vibrant Forest Brewery,"Mętne z dużą ilością goryczki, która przerasta...",torrc
29076,McEwan's Champion,5.0,+1.7,Eagle Brewery (Carlsberg Marston's Brewing Co.),Bought this beer from Tesco superstore in Nott...,ng51guy


In [62]:
locations_df

,locations,latitude,longitude,country
0,"Bungay, Suffolk",52.456225,1.437341,United Kingdom
1,"Faversham, Kent",51.314409,0.891189,United Kingdom
2,"Bristol, Gloucestershire",51.852767,-2.255369,United Kingdom
3,"Round Rock, Texas",30.508235,-97.678893,United States of America
4,"Brooklyn, New York",40.650104,-73.949582,United States of America
...,...,...,...,...
2376,"Kettering, Ohio",39.689504,-84.168827,United States of America
2377,"Jersey City, New Jersey",40.728158,-74.077642,United States of America
2378,"Hoboken, New Jersey",40.743307,-74.032375,United States of America
2379,"London, Greater London",51.507322,-0.127647,United Kingdom


In [63]:
geolocator = Nominatim(user_agent="None")

In [64]:
locations_new = pd.DataFrame(df.brewery_location[~df.brewery_location.isin(locations_df.locations)].unique(),columns=['locations'])
print(len(locations_new), "new locations to search for! Will take", len(locations_new)/2, "seconds to run.")

19 new locations to search for! Will take 9.5 seconds to run.


In [65]:
def tryconvert(x):
        try:
            return geolocator.geocode(x).latitude
        except:
            return None
    

In [66]:
def tryconvertlong(x):
        try:
            return geolocator.geocode(x).longitude
        except:
            return None

In [67]:
locations_new["latitude"] = locations_new.locations.apply(lambda x: tryconvert(x))

In [68]:
locations_new["longitude"] = locations_new.locations.apply(lambda x: tryconvertlong(x))

In [69]:
countries = []
for row in locations_new.index:
    try:
        loc = geolocator.reverse([locations_new.iloc[row,1],locations_new.iloc[row,2]], language='en')
        countries.append(loc.raw['address']['country'])
    except:
        countries.append(None)
        
    

In [70]:
locations_new['country'] = countries

In [71]:
locations_new

,locations,latitude,longitude,country
0,"West Kelowna, British Columbia",49.857464,-119.580688,Canada
1,"Merritt, British Columbia",50.111704,-120.788423,Canada
2,"Chesham, Buckinghamshire",51.706291,-0.612311,United Kingdom
3,"Wallingford, Connecticut",41.457042,-72.823155,United States of America
4,"Kalona, Iowa",41.483072,-91.706002,United States of America
5,"Laurel, Maryland",39.099275,-76.848306,United States of America
6,"Maria Stein, Ohio",40.407825,-84.493286,United States of America
7,"West Reading, Pennsylvania",40.333704,-75.947432,United States of America
8,"Brantford, Ontario",43.140816,-80.263173,Canada
9,"Berchtesgaden, Bavaria",47.631080,13.002163,Germany


In [72]:
locations_df = pd.concat([locations_df,locations_new])
locations_df.to_csv('locations_geotagged.csv',index=False)

## Merging and cleaning

In [73]:
beer_df

,return_string,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number,name_found
0,St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...,🇬🇧,Bitter - Ordinary / Best,4.3,49,95,3.25,415.0,St. Peter's Ruby Red Ale
1,Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...,🇬🇧,Bitter - Ordinary / Best,4.5,34,59,3.05,1083.0,Shepherd Neame Spitfire (Bottle)
2,Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...,🇬🇧,IPA,6.0,95,97,3.74,59.0,Arbor Rocketman
3,Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...,🇬🇧,IPA - English,6.1,46,47,3.22,420.0,Shepherd Neame India Pale Ale (Bottle)
4,Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...,🇬🇧,Strong Ale - English,7.0,45,46,3.18,409.0,Shepherd Neame Christmas Ale
...,...,...,...,...,...,...,...,...,...
15597,FrauGruber / Sudden Death / BrewHeart All Toge...,🇩🇪,IPA - Hazy / NEIPA,6.5,-,-,3.20,1.0,FrauGruber / Sudden Death / BrewHeart All Toge...
15598,Toppling Goliath Pseudo Sue - Citra Double Dry...,🇺🇸,Pale Ale - American / APA,5.8,99,100,3.98,195.0,Toppling Goliath Pseudo Sue - Citra Double Dry...
15599,Toppling Goliath Light Speed Pale Ale - Double...,🇺🇸,Pale Ale - American / APA,6.2,94,98,3.74,25.0,Toppling Goliath Light Speed Pale Ale - Double...
15600,Diamond Bear Pig Trail Porter\n🇺🇸 Porter0\n-\n...,🇺🇸,Porter,0.0,-,-,3.49,12.0,Diamond Bear Pig Trail Porter


In [74]:
# Creating DF with all details
df_all = pd.merge(df,locations_df,how='left',left_on='brewery_location',right_on='locations')
df_all = pd.merge(df_all, beer_df, how='left', left_on='beer_name',right_on='name_found')
df_all


,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name,locations,latitude,longitude,country,return_string,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number,name_found
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit,"Bungay, Suffolk",52.456225,1.437341,United Kingdom,St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...,🇬🇧,Bitter - Ordinary / Best,4.3,49,95,3.25,415.0,St. Peter's Ruby Red Ale
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray,"Faversham, Kent",51.314409,0.891189,United Kingdom,Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...,🇬🇧,Bitter - Ordinary / Best,4.5,34,59,3.05,1083.0,Shepherd Neame Spitfire (Bottle)
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG,"Bristol, Gloucestershire",51.852767,-2.255369,United Kingdom,Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...,🇬🇧,IPA,6.0,95,97,3.74,59.0,Arbor Rocketman
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit,"Faversham, Kent",51.314409,0.891189,United Kingdom,Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...,🇬🇧,IPA - English,6.1,46,47,3.22,420.0,Shepherd Neame India Pale Ale (Bottle)
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit,"Faversham, Kent",51.314409,0.891189,United Kingdom,Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...,🇬🇧,Strong Ale - English,7.0,45,46,3.18,409.0,Shepherd Neame Christmas Ale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29351,Miller High Life,4.0,+2.3,Miller Brewing Company (Molson Coors),"People can trash this beer all they want, but ...",moomin,+2.3,51.254303,7.148721,Germany,Miller High Life\n🇺🇸 Pale Lager - North Am. / ...,🇺🇸,Pale Lager - North Am. / Light / Rice,4.6,0,9,1.94,1832.0,Miller High Life
29352,New Glarus Spotted Cow,4.7,+1.6,New Glarus Brewing Company,"A classic delicious Wisconsin beer, maybe my f...",moomin,+1.6,48.718182,9.989251,Germany,New Glarus Spotted Cow\n🇺🇸 Saison / Farmhouse ...,🇺🇸,Saison / Farmhouse / Grisette,4.9,40,17,3.30,1107.0,New Glarus Spotted Cow
29353,Vibrant Forest Stone Sap - Juicy India Pale Ale,1.9,-1.6,Vibrant Forest Brewery,"Mętne z dużą ilością goryczki, która przerasta...",torrc,-1.6,48.718182,9.989251,Germany,Vibrant Forest Stone Sap - Juicy India Pale Al...,🇬🇧,IPA - Hazy / NEIPA,6.8,85,47,3.47,33.0,Vibrant Forest Stone Sap - Juicy India Pale Ale
29354,McEwan's Champion,5.0,+1.7,Eagle Brewery (Carlsberg Marston's Brewing Co.),Bought this beer from Tesco superstore in Nott...,ng51guy,+1.7,48.780313,10.039726,Germany,McEwan's Champion\n🇬🇧 Scotch Ale / Wee Heavy •...,🇬🇧,Scotch Ale / Wee Heavy,7.3,55,41,3.36,320.0,McEwan's Champion


In [75]:
# drop excess columns
df_all.drop(["locations","return_string","name_found"],axis=1,inplace=True)
# drop rows with ratings instead of location
df_all = df_all[~df_all['brewery_location'].str.len()<4]
# drop all NaNs
print(len(df_all) - len(df_all.dropna()), "columns with NaNs dropped.")
df_all.dropna(inplace=True)


2195 columns with NaNs dropped.


In [76]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27161 entries, 0 to 29355
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   beer_name         27161 non-null  object 
 1   beer_rating       27161 non-null  float64
 2   brewery_location  27161 non-null  object 
 3   brewery_name      27161 non-null  object 
 4   review_content    27161 non-null  object 
 5   reviewer_name     27161 non-null  object 
 6   latitude          27161 non-null  float64
 7   longitude         27161 non-null  float64
 8   country           27161 non-null  object 
 9   flag              27161 non-null  object 
 10  type              27161 non-null  object 
 11  abv               27161 non-null  float64
 12  rb_overall_score  27161 non-null  object 
 13  rb_style_score    27161 non-null  object 
 14  rb_user_rating    27161 non-null  object 
 15  rb_rating_number  27161 non-null  float64
dtypes: float64(5), object(11)
memory usage: 

## Deal with Non-English Descriptions

## Save down cleaned data

In [77]:
df_all.to_csv("data_clean.csv",index=False)